<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/Z_image_turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Z-image-による画像生成

このコードへのリンクは[こちら](https://j.aicu.ai/CZimg) 解説はこちら [https://j.aicu.ai/251127](https://j.aicu.ai/251127)


In [ ]:
# @title 1. ライブラリのインストール（数分かかるよ！）
# 最新のZ-Image対応diffusersを入れるためにgitからインストールします
!pip install git+https://github.com/huggingface/diffusers
!pip install accelerate sentencepiece protobuf

In [ ]:
# @title 2. モデルの読み込み
import torch
from diffusers import ZImagePipeline

# メモリ節約のために bfloat16 を使うのがコツだよ！
dtype = torch.bfloat16

print("モデルをダウンロード中...（ちょっと待ってね）")
pipe = ZImagePipeline.from_pretrained(
    "Tongyi-MAI/Z-Image-Turbo",
    torch_dtype=dtype,
    low_cpu_mem_usage=False, # クラッシュする場合はTrueにしてみて
)

# ColabのT4 GPU (16GB) で動かすための設定
# もしVRAM不足でエラーが出る場合は、下の pipe.to("cuda") をコメントアウトして
# 代わりに pipe.enable_model_cpu_offload() の「#」を外してね！

pipe.to("cuda")
# pipe.enable_model_cpu_offload() # VRAMがキツい時はこっち！

print("準備完了！✨")

In [ ]:
# @title 3. 画像生成（8ステップで爆速！）
import torch
from IPython.display import display

# メイのおすすめプロンプト（公式例）
prompt = "Young Chinese woman in red Hanfu, intricate embroidery. Impeccable makeup, red floral forehead pattern. Elaborate high bun, golden phoenix headdress, red flowers, beads. Holds round folding fan with lady, trees, bird. Neon lightning-bolt lamp (⚡️), bright yellow glow, above extended left palm. Soft-lit outdoor night background, silhouetted tiered pagoda (西安大雁塔), blurred colorful distant lights."

# 生成の設定
# Turboモデルは guidance_scale=0.0 が推奨！
# num_inference_steps=9 で実質8回の計算になるんだって！
image = pipe(
    prompt=prompt,
    height=1024,
    width=1024,
    num_inference_steps=9,
    guidance_scale=0.0,
    generator=torch.Generator("cuda").manual_seed(42), # 乱数シードを固定
).images[0]

# 画像を表示＆保存
image.save("z_image_result.png")
display(image)

In [ ]:
# @title 1. 日本語プロンプトで16:9生成 (1280x720)
import torch

# ユーザーさんが成功したプロンプト
prompt_jp_fixed = "紺の和服を着た若く美しい日本の男性、完璧なメイク、顔の横にネオンの稲妻ランプ（⚡️）があり、明るい黄色の輝きを放っている。柔らかな照明の屋外の夜の背景、シルエットになった東京タワー、ぼやけた色とりどりの遠くの明かり。"

print(f"Generating: {prompt_jp_fixed}")

image_wide = pipe(
    prompt=prompt_jp_fixed,
    height=720,  # 縦
    width=1280,  # 横 (16:9)
    num_inference_steps=9,
    guidance_scale=0.0,
    generator=torch.Generator("cuda").manual_seed(123), # シードはお好みで変更してね
).images[0]

# 保存と表示
image_wide.save("z_image_wide_jp.png")
display(image_wide)

In [ ]:
# @title 2. キーワード・ガチャ！ (日本語 & 英語) 9:16生成
import random
from PIL import Image

# 画像を横に連結する関数 (PIL使用)
def concat_images_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

# --- キーワードリストの定義 ---

# 共通部分（固定）
fixed_part_jp = "顔の横にネオンの稲妻ランプ（⚡️）があり、明るい黄色の輝きを放っている。"
fixed_part_en = "Neon lightning bolt lamp (⚡️) next to the face, emitting a bright yellow glow."

# ガチャ要素
options = {
    "outfit": {
        "jp": ["紺の和服", "初音ミクのコスプレ", "アイルランドの民族衣装"],
        "en": ["Dark blue kimono", "Hatsune Miku cosplay", "Irish folk costume"]
    },
    "subject_adj": {
        "jp": ["若く美しい"],
        "en": ["Young beautiful"]
    },
    "nationality": {
        "jp": ["日本", "中国", "韓国", "フランス", "アメリカ", "アフリカ"],
        "en": ["Japanese", "Chinese", "Korean", "French", "American", "African"]
    },
    "gender": {
        "jp": ["男性", "女性"],
        "en": ["male", "female"]
    },
    "condition": {
        "jp": ["完璧なメイク", "泥まみれ"],
        "en": ["perfect makeup", "covered in mud"]
    },
    "setting": {
        "jp": ["柔らかな照明の屋外の夜", "観光地", "朝の室内"],
        "en": ["Soft outdoor night lighting", "Tourist spot", "Morning indoor"]
    },
    "bg_object": {
        "jp": ["シルエットになった東京タワー", "シルエットになったエッフェル塔", "シルエットになった自由の女神"],
        "en": ["silhouetted Tokyo Tower", "silhouetted Eiffel Tower", "silhouetted Statue of Liberty"]
    },
    "bg_detail": {
        "jp": ["ぼやけた色とりどりの遠くの明かり", "野生の動物"],
        "en": ["blurred colorful distant lights", "wild animals"]
    }
}

# --- プロンプト生成関数 ---
def build_random_prompt(lang="jp"):
    # ランダムに選択
    outfit = random.choice(options["outfit"][lang])
    adj = random.choice(options["subject_adj"][lang])
    nat = random.choice(options["nationality"][lang])
    gen = random.choice(options["gender"][lang])
    cond = random.choice(options["condition"][lang])
    sett = random.choice(options["setting"][lang])
    bg_obj = random.choice(options["bg_object"][lang])
    bg_det = random.choice(options["bg_detail"][lang])

    # 文章を組み立てる
    if lang == "jp":
        prompt = f"{outfit}を着た{adj}{nat}の{gen}、{cond}、{fixed_part_jp} {sett}の背景、{bg_obj}、{bg_det}。"
    else:
        prompt = f"{adj} {nat} {gen} wearing {outfit}, {cond}, {fixed_part_en} Background of {sett}, {bg_obj}, {bg_det}."

    return prompt

# --- 生成実行 ---

# 1. 日本語プロンプトで生成
prompt_jp = build_random_prompt("jp")
print(f"🇯🇵 JP Prompt: {prompt_jp}")
image_jp = pipe(
    prompt=prompt_jp,
    height=1280, width=720, # 9:16
    num_inference_steps=9,
    guidance_scale=0.0,
    generator=torch.Generator("cuda").manual_seed(random.randint(0, 10000)),
).images[0]

# 2. 英語プロンプトで生成（別の組み合わせになります）
prompt_en = build_random_prompt("en")
print(f"🇺🇸 EN Prompt: {prompt_en}")
image_en = pipe(
    prompt=prompt_en,
    height=1280, width=720, # 9:16
    num_inference_steps=9,
    guidance_scale=0.0,
    generator=torch.Generator("cuda").manual_seed(random.randint(0, 10000)),
).images[0]

# 3. 横に並べて表示
print("\n✨ Left: Japanese Prompt / Right: English Prompt")
combined_image = concat_images_h(image_jp, image_en)
display(combined_image)

# 保存
combined_image.save("z_image_random_mix.png")

In [ ]:
# @title 🔥 全組み合わせ生成（シャッフル版！毎回違うキャラが出ます）
import os
import itertools
import torch
import time
import datetime
import random # シャッフル用
from PIL import Image
from google.colab import drive
from tqdm import tqdm

# 1. Google Driveをマウント
if not os.path.exists('/content/drive'):
    print("📂 Google Driveをマウントします...")
    drive.mount('/content/drive')

# 保存先ディレクトリ
output_dir = "/content/drive/MyDrive/z-image-generated-shuffle" # 混ざらないようにフォルダ変えました
os.makedirs(output_dir, exist_ok=True)
print(f"📂 保存先: {output_dir}")

# ログファイル
log_file_path = os.path.join(output_dir, "z-image-prompt-shuffle.txt")

# 2. キーワード定義
outfits_jp = ["紺の和服", "初音ミクのコスプレ", "アイルランドの民族衣装"]
outfits_en = ["Dark blue kimono", "Hatsune Miku cosplay", "Irish folk costume"]

nationalities_jp = ["日本", "中国", "韓国", "フランス", "アメリカ", "アフリカ"]
nationalities_en = ["Japanese", "Chinese", "Korean", "French", "American", "African"]

genders_jp = ["男性", "女性"]
genders_en = ["male", "female"]

conditions_jp = ["完璧なメイク", "泥まみれ"]
conditions_en = ["perfect makeup", "covered in mud"]

settings_jp = ["柔らかな照明の屋外の夜", "観光地", "朝の室内"]
settings_en = ["Soft outdoor night lighting", "Tourist spot", "Morning indoor"]

bg_objects_jp = ["シルエットになった東京タワー", "シルエットになったエッフェル塔", "シルエットになった自由の女神"]
bg_objects_en = ["silhouetted Tokyo Tower", "silhouetted Eiffel Tower", "silhouetted Statue of Liberty"]

bg_details_jp = ["ぼやけた色とりどりの遠くの明かり", "野生の動物"]
bg_details_en = ["blurred colorful distant lights", "wild animals"]

# 固定パーツ
fixed_jp = "若く美しい"
fixed_en = "Young beautiful"
lightning_jp = "顔の横にネオンの稲妻ランプ（⚡️）があり、明るい黄色の輝きを放っている。"
lightning_en = "Neon lightning bolt lamp (⚡️) next to the face, emitting a bright yellow glow."

# 画像連結関数
def concat_images_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

# 3. 全組み合わせを作成
combinations = list(itertools.product(
    range(len(outfits_jp)),      # 0
    range(len(nationalities_jp)),# 1
    range(len(genders_jp)),      # 2
    range(len(conditions_jp)),   # 3
    range(len(settings_jp)),     # 4
    range(len(bg_objects_jp)),   # 5
    range(len(bg_details_jp))    # 6
))

# 🔥 ここがポイント！リストをシャッフルします 🔥
random.seed(42) # 再現性のためシード固定（いつ実行しても同じランダム順になる）
random.shuffle(combinations)

total_count = len(combinations)
print(f"⚡️ 生成予定: {total_count} パターン (ランダム順で生成します！)")

# 4. ループ実行
with open(log_file_path, "a", encoding="utf-8") as f:
    start_time_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    f.write(f"\n{'='*20}\n🚀 Random Shuffle Generation Started at {start_time_str}\nTotal Patterns: {total_count}\n{'='*20}\n")

for i, idxs in enumerate(tqdm(combinations, desc="Generating")):
    i_out, i_nat, i_gen, i_cond, i_set, i_bg_obj, i_bg_det = idxs

    # プロンプト構築
    p_jp = f"{outfits_jp[i_out]}を着た{fixed_jp}、{nationalities_jp[i_nat]}の{genders_jp[i_gen]}、{conditions_jp[i_cond]}、{lightning_jp} {settings_jp[i_set]}の背景、{bg_objects_jp[i_bg_obj]}、{bg_details_jp[i_bg_det]}。"
    p_en = f"{fixed_en} {nationalities_en[i_nat]} {genders_en[i_gen]} wearing {outfits_en[i_out]}, {conditions_en[i_cond]}, {lightning_en} Background of {settings_en[i_set]}, {bg_objects_en[i_bg_obj]}, {bg_details_en[i_bg_det]}."

    # シード値は生成順序(i)に基づいて変化させる
    seed = 20000 + i

    try:
        # --- 時間計測開始 ---
        start_time = time.time()

        # JP生成
        img_jp = pipe(
            prompt=p_jp,
            height=1280, width=720,
            num_inference_steps=9,
            guidance_scale=0.0,
            generator=torch.Generator("cuda").manual_seed(seed),
        ).images[0]

        # EN生成
        img_en = pipe(
            prompt=p_en,
            height=1280, width=720,
            num_inference_steps=9,
            guidance_scale=0.0,
            generator=torch.Generator("cuda").manual_seed(seed),
        ).images[0]

        elapsed_time = time.time() - start_time
        # --- 時間計測終了 ---

        # 結合
        combined = concat_images_h(img_jp, img_en)

        # 保存
        base_name = f"z_shuffle_{i:04d}"
        img_save_path = os.path.join(output_dir, f"{base_name}.png")
        combined.save(img_save_path)

        # ログ追記
        log_entry = (
            f"[{i:04d}/{total_count}] Seed: {seed} | Time: {elapsed_time:.2f}s\n"
            f"File: {base_name}.png\n"
            f"JP: {p_jp}\n"
            f"EN: {p_en}\n"
            f"{'-'*40}\n"
        )
        with open(log_file_path, "a", encoding="utf-8") as f:
            f.write(log_entry)

    except Exception as e:
        error_msg = f"❌ Error at index {i}: {e}\n"
        print(error_msg)
        with open(log_file_path, "a", encoding="utf-8") as f:
            f.write(error_msg)
        continue

print(f"\n✅ 完了！画像は '{output_dir}' を確認してね！")